# Gaussian Elimination


## In general

The system of linar equations $Ax=b$
is solved in three steps (_without pivoting_ ):

1. $A=LU\qquad$ (LU factorization, $O(\frac{2}{3}n^3)$ operations),
2. $Ly=b\qquad$ (solving lower triangular system, $n^2$ operations),
3. $Ux=y\qquad$ (solving upper triangular system, $n^2$ operations).

With pivoting we have

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## Examples

The following class of problems gives rise to two separate types of issues, one we have already discussed, one we have not. Below  $\epsilon$ is the value generated by the function `eps()`.

Consider the system of linear equations 

\begin{eqnarray*}
\displaystyle\frac{\epsilon}{10} x_1 + x_2 = 1 \\
x_1 + x_2 = 2
\end{eqnarray*}

A good approximate answer is $x_1 = x_2 =1$. Use the augmented system approach:

\begin{align*}
&\left(\begin{array}{cc|c} \displaystyle\displaystyle\frac{\epsilon}{10} & 1 & 1 \\ 1 & 1 & 2 \end{array} \right) \sim
\left(\begin{array}{cc|c} \displaystyle \displaystyle\frac{\epsilon}{10} & 1 & 1 \\ 0 & 1-\displaystyle\displaystyle\frac{10}{\epsilon} & 2-\displaystyle\displaystyle\frac{10}{\epsilon}\end{array}\right)
\approx \left(\begin{array}{cc|c}  \displaystyle\displaystyle\frac{\epsilon}{10} & 1 & 1 \\ 0 & -\displaystyle\displaystyle\frac{10}{\epsilon} & -\displaystyle\displaystyle\frac{10}{\epsilon}\end{array}\right).
\end{align*}

The last transformation is rounding to the machine precision $\epsilon$.
The very significant "1" and "2" are _rounded away_ in the last line!

Back solve to get $x_1 = 0$, $x_2 = 1$. If you put these values back in the orgininal system, note that $x_1+x_2 = 1$, so this is "way off."

In [1]:
[eps()/10 1;0 1-10/eps()]\[1; 2-10/eps()]

2-element Array{Float64,1}:
 0.0
 1.0

Again there is a "fix", it is called _partial pivoting_. Put largest uneliminated entry in the column in pivot or diagonal position :

\begin{align*}
&\left(\begin{array}{cc|c}     1 & 1 & 2 \\\displaystyle\frac{\epsilon}{10} & 1 & 1 \end{array}\right) \sim
\left(\begin{array}{cc|c}     1 & 1 & 2 \\0                   & 1-\displaystyle\frac{\epsilon}{10}&1-\displaystyle\frac{\epsilon}{10}\end{array}\right)
\approx   \left(\begin{array}{cc|c}     1 & 1 & 2 \\0                   & 1                    &1                    \end{array}\right)
\end{align*}

In [2]:
[1 1;0 1-eps()/10]\[2; 1-eps()/10]

2-element Array{Float64,1}:
 1.0
 1.0

This is the correct solution to machine precision.

Sometimes changing the algorithm does _no good at all_ ! Consider the system 

\begin{align*}
(1+2\epsilon)x_1 + (1+2\epsilon)x_2 = 2 \\
(1+\epsilon)x_1 + x_2 =2
\end{align*}

Using the augmented matrix approach

$$
\left(\begin{array}{cc|c}
(1+2\epsilon)&     (1+2\epsilon )&     2 \\
(1+\epsilon)&     1   &2 \end{array}\right).
$$

Mulitply the first row by $\alpha = (1+\epsilon)/(1+2\epsilon)= 1-\epsilon + O(\epsilon^2)$
and add to the second row:

$$
\left(\begin{array}{cc|c}
(1+2\epsilon)&     (1+2\epsilon )&     2 \\
0           &     -\epsilon & 2\epsilon \end{array}\right).
$$

The solution is $x_1 = 4$ and $x_2 =-2$. This is correct to machine precision.

A small change in the right hand side yields

$$
\left(\begin{array}{cc|c}
(1+2\epsilon)&     (1+2\epsilon )&     2+4\epsilon \\
(1+\epsilon)&     1   &2 +\epsilon \end{array}\right).
$$

The correct answer is $x_1=x_2=1$, but with rounding we get  $x_1 =0$, $x_2 =2$. __Explain.__
Every trick I know except increasing the precision, yields similar wrong answers. 

In [3]:
[1+2*eps() 1+2*eps(); 1+eps() 1]\[2+4*eps(); 2+eps()]

2-element Array{Float64,1}:
 0.0
 2.0

In [4]:
[BigFloat(1)+2*eps() 1+2*eps(); 1+eps() 1]\[BigFloat(2)+4*eps(); 2+eps()]

2-element Array{BigFloat,1}:
 0.0
 2.0

__Razlog.__   IEEE Arithmetic rounds this system to

 $$
\left(\begin{array}{cc|c}
(1+2\epsilon)&     (1+2\epsilon )&     2+4\epsilon \\
(1+\epsilon)&     1   &2           \end{array}\right)
$$

which has the solution $x_1=0$ and $x_2=2$. This problem is very close to the singular system

$$
\left(\begin{array}{cc|c} 1 & 1 & 2 \\ 1 & 1 & 2\end{array}\right)
$$

which has the parametric solutions

$$
\mathbf{x} =\begin{pmatrix} x_1 \\ x_2\end{pmatrix} = \begin{pmatrix} 1\\1\end{pmatrix}+ 
\beta \begin{pmatrix}-1 \\1\end{pmatrix}, \quad \beta \in \mathbb{R}.
$$

Notice that $\begin{pmatrix} x_1 \\ x_2\end{pmatrix}= \begin{pmatrix} 1\\1\end{pmatrix}$ i $\begin{pmatrix} x_1 \\ x_2\end{pmatrix}=\begin{pmatrix} 0\\2\end{pmatrix}$ are two of those solutions.

__Question.__ What is the geometric interpretation of this system?

## LU factorization

In [5]:
function mylu(A₁::Array{T}) where T # Strang, str. 100
    A=copy(A₁)
    n,m=size(A)
    # This accepts numbers and block matrices
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [6]:
using LinearAlgebra
import Random
Random.seed!(123)
A=rand(6,6)

6×6 Array{Float64,2}:
 0.768448  0.586022   0.865412  0.582318  0.20923   0.48
 0.940515  0.0521332  0.617492  0.255981  0.918165  0.790201
 0.673959  0.26864    0.285698  0.70586   0.614255  0.356221
 0.395453  0.108871   0.463847  0.291978  0.802665  0.900925
 0.313244  0.163666   0.275819  0.281066  0.555668  0.529253
 0.662555  0.473017   0.446568  0.792931  0.940782  0.031831

In [7]:
L,U=mylu(A);

In [8]:
L

6×6 Array{Float64,2}:
 1.0       0.0         0.0         0.0        0.0     0.0
 1.22392   1.0         0.0         0.0        0.0     0.0
 0.877039  0.368849    1.0         0.0        0.0     0.0
 0.514613  0.289733   -0.471902    1.0        0.0     0.0
 0.407632  0.113088    0.0869892   0.215088   1.0     0.0
 0.862199  0.0484897   0.896224   -0.0434479  2.3274  1.0

In [9]:
U

6×6 Array{Float64,2}:
 0.768448   0.586022   0.865412   0.582318  0.20923    0.48
 0.0       -0.665108  -0.441699  -0.456726  0.662085   0.202722
 0.0        0.0       -0.310382   0.363608  0.186542  -0.139531
 0.0        0.0        0.0        0.296226  0.591194   0.52933
 0.0        0.0        0.0        0.0       0.25212    0.20895
 0.0        0.0        0.0        0.0       0.0       -0.730114

In [10]:
L*U-A

6×6 Array{Float64,2}:
 0.0  0.0          0.0          0.0  0.0   0.0
 0.0  0.0          0.0          0.0  0.0   0.0
 0.0  0.0          5.55112e-17  0.0  0.0   0.0
 0.0  1.38778e-17  0.0          0.0  0.0   0.0
 0.0  0.0          0.0          0.0  0.0  -1.11022e-16
 0.0  0.0          0.0          0.0  0.0   0.0

## Triangular systems

In [11]:
function myU(U::Array{T},b₁::Array{T}) where T
    b=copy(b₁)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL(L::Array{T},b₁::Array{T}) where T
    b=copy(b₁)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [12]:
b=rand(6)

6-element Array{Float64,1}:
 0.9006814789827005
 0.9402992421257947
 0.6213787149845327
 0.34817276542456277
 0.57061318926682
 0.20399662276026764

In [13]:
# Solve the system using the built-in function
x=A\b

6-element Array{Float64,1}:
  2.724177128392811
  4.635862756193173
 -3.511396551245912
 -2.6333845620867318
 -0.19586385967020775
  1.4663093725368699

In [14]:
# Solve the system using our functions
y=myL(L,b)

6-element Array{Float64,1}:
  0.9006814789827005
 -0.16205875724336183
 -0.10877893946554916
 -0.11970879824874248
  0.25700386570333467
 -1.0705725308278367

In [15]:
x₁=myU(U,y)

6-element Array{Float64,1}:
  2.724177128392812
  4.635862756193177
 -3.5113965512459138
 -2.6333845620867335
 -0.1958638596702071
  1.46630937253687

In [16]:
# Compare the solutions
x-x₁

6-element Array{Float64,1}:
 -8.881784197001252e-16
 -3.552713678800501e-15
  1.7763568394002505e-15
  1.7763568394002505e-15
 -6.38378239159465e-16
 -2.220446049250313e-16

## Speed

The function `mylu()` is slow. Among other reasons, it allocates unnecessarily three matrices and it does not work with block matrices.

The function can be reformulated such that $L$ and $U$ are stored in the array $A$, where the diagonal  of $L$ is not stored since all elements are 1 (see [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [17]:
function mylu₁(A₁::Array{T}) where T # Strang, str. 100
    A=copy(A₁)
    n,m=size(A)
    for k=1:n-1
        ρ=k+1:n
        A[ρ,k]=A[ρ,k]/A[k,k]
        A[ρ,ρ]=A[ρ,ρ]-A[ρ,k]*A[k,ρ]'
    end
    A
end

mylu₁ (generic function with 1 method)

In [18]:
mylu₁(A)

6×6 Array{Float64,2}:
 0.768448   0.586022    0.865412    0.582318   0.20923    0.48
 1.22392   -0.665108   -0.441699   -0.456726   0.662085   0.202722
 0.877039   0.368849   -0.310382    0.363608   0.186542  -0.139531
 0.514613   0.289733   -0.471902    0.296226   0.591194   0.52933
 0.407632   0.113088    0.0869892   0.215088   0.25212    0.20895
 0.862199   0.0484897   0.896224   -0.0434479  2.3274    -0.730114

In [19]:
L

6×6 Array{Float64,2}:
 1.0       0.0         0.0         0.0        0.0     0.0
 1.22392   1.0         0.0         0.0        0.0     0.0
 0.877039  0.368849    1.0         0.0        0.0     0.0
 0.514613  0.289733   -0.471902    1.0        0.0     0.0
 0.407632  0.113088    0.0869892   0.215088   1.0     0.0
 0.862199  0.0484897   0.896224   -0.0434479  2.3274  1.0

In [20]:
U

6×6 Array{Float64,2}:
 0.768448   0.586022   0.865412   0.582318  0.20923    0.48
 0.0       -0.665108  -0.441699  -0.456726  0.662085   0.202722
 0.0        0.0       -0.310382   0.363608  0.186542  -0.139531
 0.0        0.0        0.0        0.296226  0.591194   0.52933
 0.0        0.0        0.0        0.0       0.25212    0.20895
 0.0        0.0        0.0        0.0       0.0       -0.730114

Compare execution times of LAPACK-based function `lu()` and our naïve function `mylu()` on larger matrix. 

Execute several times for more accurate timings.

In [21]:
n=512
A=rand(n,n);

In [22]:
@time lu(A);

  0.020967 seconds (94 allocations: 2.009 MiB)


In [23]:
@time mylu₁(A);

  0.733275 seconds (5.49 k allocations: 1.003 GiB, 31.56% gc time)


### Block variant

`mylu()` and `mylu\_1()` are tens of times slower than `lu()`.

Let us rewrite `mylu\_1()` to work with blocks (still there is no pivoting!):

In [24]:
function mylu₂(A₁::Array{T}) where T # Strang, page 100
    A=copy(A₁)
    n,m=size(A)
    for k=1:n-1
        for ρ=k+1:n
            A[ρ,k]=A[ρ,k]/A[k,k]
            for l=k+1:n
                A[ρ,l]=A[ρ,l]-A[ρ,k]*A[k,l]
            end
        end
    end
    A
end

mylu₂ (generic function with 1 method)

First the small test:

In [25]:
k,l=2,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [26]:
A₀=mylu₂(Ab)

4×4 Array{Array{Float64,2},2}:
 [0.19178 0.0976698; 0.234544 0.627093]  …  [0.295925 0.164099; 0.942843 0.830942]
 [5.80254 -0.70825; 3.25685 -0.331261]      [-0.339473 -0.265039; -0.232242 0.107999]
 [3.23452 -0.450395; 2.34086 0.764213]      [0.0241067 1.07794; -0.633465 0.531221]
 [0.838168 0.600568; 0.876648 0.608517]     [-0.124081 0.60866; -0.886185 0.597327]

In [27]:
# Check
U=triu(A₀)
L=tril(A₀)
for i=1:maximum(size(L))
    L[i,i]=Matrix{Float64}(I,size(L[1,1])) # eye(L[1,1])
end

In [28]:
Residual=L*U-Ab

4×4 Array{Array{Float64,2},2}:
 [0.0 0.0; 0.0 0.0]                    …  [0.0 0.0; 0.0 0.0]
 [0.0 -1.11022e-16; 0.0 -2.77556e-17]     [0.0 0.0; 0.0 0.0]
 [0.0 0.0; 0.0 0.0]                       [0.0 0.0; 0.0 0.0]
 [0.0 -5.55112e-17; 0.0 -5.55112e-17]     [0.0 0.0; 0.0 0.0]

In [29]:
# Converting block matrix into a standard one
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) 
        for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [30]:
norm(unblock(Residual))

3.444376352465766e-16

Try larger dimensions ($n=k\cdot l$).

In [31]:
k,l=64,8 # 64, 8
Ab=[rand(k,k) for i=1:l, j=1:l];

In [32]:
@time mylu₂(Ab);

  0.050656 seconds (841 allocations: 11.422 MiB)


We see that `mylu\_2()` is nearly as fast as `lu()` (on one core), with the remark that `mylu\_2()` uses no pivoting. The function is still not optimal since it allocates to much memory.

## Pivoting

Standard implementations always compute Gaussian elimination using _partial pivoting_ :

In each step, the rows are reordered (interchanged) such that the pivot element has largest absolute value in the active part of the given column. As a consequence

$$|L_{ij}| \leq 1,$$

which sufficiently reduces element growth (in practice).

In [33]:
A=[0.00003 1;2 3]

2×2 Array{Float64,2}:
 3.0e-5  1.0
 2.0     3.0

In [34]:
L,U=mylu(A)
L

2×2 Array{Float64,2}:
     1.0  0.0
 66666.7  1.0

In [35]:
U

2×2 Array{Float64,2}:
 3.0e-5       1.0
 0.0     -66663.7

In [36]:
# With pivoting
P=[0 1;1 0]
L,U=mylu(P*A)
L

2×2 Array{Float64,2}:
 1.0     0.0
 1.5e-5  1.0

In [37]:
U

2×2 Array{Float64,2}:
 2.0  3.0
 0.0  0.999955

In [38]:
L*U-P*A

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

Now the built-in function. Use it precisely.

In [39]:
?lu

search: lu lu! LU flush mylu mylu₂ mylu₁ values include include_string



```
lu(A, pivot=Val(true); check = true) -> F::LU
```

Compute the LU factorization of `A`.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

In most cases, if `A` is a subtype `S` of `AbstractMatrix{T}` with an element type `T` supporting `+`, `-`, `*` and `/`, the return type is `LU{T,S{T}}`. If pivoting is chosen (default) the element type should also support `abs` and `<`.

The individual components of the factorization `F` can be accessed via `getproperty`:

| Component | Description                         |
|:--------- |:----------------------------------- |
| `F.L`     | `L` (lower triangular) part of `LU` |
| `F.U`     | `U` (upper triangular) part of `LU` |
| `F.p`     | (right) permutation `Vector`        |
| `F.P`     | (right) permutation `Matrix`        |

Iterating the factorization produces the components `F.L`, `F.U`, and `F.p`.

The relationship between `F` and `A` is

`F.L*F.U == A[F.p, :]`

`F` further supports the following functions:

| Supported function  | `LU` | `LU{T,Tridiagonal{T}}` |
|:------------------- |:---- |:---------------------- |
| [`/`](@ref)         | ✓    |                        |
| [`\`](@ref)         | ✓    | ✓                      |
| [`inv`](@ref)       | ✓    | ✓                      |
| [`det`](@ref)       | ✓    | ✓                      |
| [`logdet`](@ref)    | ✓    | ✓                      |
| [`logabsdet`](@ref) | ✓    | ✓                      |
| [`size`](@ref)      | ✓    | ✓                      |

# Examples

```jldoctest
julia> A = [4 3; 6 3]
2×2 Array{Int64,2}:
 4  3
 6  3

julia> F = lu(A)
LU{Float64,Array{Float64,2}}
L factor:
2×2 Array{Float64,2}:
 1.0       0.0
 0.666667  1.0
U factor:
2×2 Array{Float64,2}:
 6.0  3.0
 0.0  1.0

julia> F.L * F.U == A[F.p, :]
true

julia> l, u, p = lu(A); # destructuring via iteration

julia> l == F.L && u == F.U && p == F.p
true
```

---

```
lu(A::SparseMatrixCSC; check = true) -> F::UmfpackLU
```

Compute the LU factorization of a sparse matrix `A`.

For sparse `A` with real or complex element type, the return type of `F` is `UmfpackLU{Tv, Ti}`, with `Tv` = [`Float64`](@ref) or `ComplexF64` respectively and `Ti` is an integer type ([`Int32`](@ref) or [`Int64`](@ref)).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

The individual components of the factorization `F` can be accessed by indexing:

| Component | Description                         |
|:--------- |:----------------------------------- |
| `L`       | `L` (lower triangular) part of `LU` |
| `U`       | `U` (upper triangular) part of `LU` |
| `p`       | right permutation `Vector`          |
| `q`       | left permutation `Vector`           |
| `Rs`      | `Vector` of scaling factors         |
| `:`       | `(L,U,p,q,Rs)` components           |

The relation between `F` and `A` is

`F.L*F.U == (F.Rs .* A)[F.p, F.q]`

`F` further supports the following functions:

  * [`\`](@ref)
  * [`cond`](@ref)
  * [`det`](@ref)

!!! note
    `lu(A::SparseMatrixCSC)` uses the UMFPACK library that is part of SuiteSparse. As this library only supports sparse matrices with [`Float64`](@ref) or `ComplexF64` elements, `lu` converts `A` into a copy that is of type `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.



In [40]:
F=lu(A)

LU{Float64,Array{Float64,2}}
L factor:
2×2 Array{Float64,2}:
 1.0     0.0
 1.5e-5  1.0
U factor:
2×2 Array{Float64,2}:
 2.0  3.0
 0.0  0.999955

In [41]:
F.L*F.U == A[F.p, :]

true

In [42]:
# Random matrix
Random.seed!(248)
A=rand(5,5)
L,U,P=lu(A);

In [43]:
P

5-element Array{Int64,1}:
 3
 4
 5
 2
 1

In [44]:
L

5×5 Array{Float64,2}:
 1.0        0.0         0.0       0.0       0.0
 0.0820339  1.0         0.0       0.0       0.0
 0.437335   0.0959387   1.0       0.0       0.0
 0.928548   0.0219166   0.150299  1.0       0.0
 0.386654   0.74805    -0.562755  0.803388  1.0

In [45]:
U

5×5 Array{Float64,2}:
 0.740619  0.105456  0.288167  0.0134151   0.810915
 0.0       0.939039  0.427963  0.871155    0.820178
 0.0       0.0       0.697849  0.776953    0.204272
 0.0       0.0       0.0       0.649058   -0.21677
 0.0       0.0       0.0       0.0         0.183107

In [46]:
# There are rounding errors!
L*U==A[P,:]

false

In [47]:
L*U-A[P, :]

5×5 Array{Float64,2}:
 0.0  0.0           0.0           0.0          0.0
 0.0  0.0           0.0           0.0          0.0
 0.0  0.0          -1.11022e-16   0.0          0.0
 0.0  0.0           0.0           0.0          0.0
 0.0  1.11022e-16   6.93889e-18  -1.11022e-16  0.0

### Complete pivoting

The following function computes Gaussian elimination using _complete pivoting_ - in each step, rows and columns are interchanged such that the element with the largest absolute value in the current submatrix is brought to the pivot position. Theoretically, element growth is bounded, but the bound is $O(2^n)$ which is not useful. 

In [48]:
function gecp(A₁::Array{T}) where T
    # Gaussian elimination with complete pivoting
    # On exit, either Pr*L*U*Pc'=A or Pr'*A*Pc=L*U
    A=copy(A₁)
    n,m=size(A)
    Pr=Matrix{T}(I,n,n)
    Pc=Matrix{T}(I,n,n)
    D=zeros(n)
    for i=1:n-1
        amax,indm=findmax(abs.(A[i:n,i:n]))
        imax=indm[1]+i-1
        jmax=indm[2]+i-1
        # Interchanging rows
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # Interchanging columns
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # Elimination
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]'
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=I+tril(A,-1)
    U=I+triu(A,1)
    U=Diagonal(D)*U
    return L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [49]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.9665983607050483
 0.43239336629240754
 0.2991638766124043
 0.7502210485360699
 0.15514720539742077

In [50]:
L,U,Pr,Pc=gecp(A);

In [51]:
Pr

5×5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [52]:
Pr*L*U*Pc'-A

5×5 Array{Float64,2}:
 0.0   5.20417e-18  0.0  0.0   0.0
 0.0   0.0          0.0  0.0   0.0
 0.0  -5.55112e-17  0.0  0.0   0.0
 0.0   0.0          0.0  0.0   0.0
 0.0   0.0          0.0  0.0  -1.11022e-16

In [53]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.43239336629240754
  0.4176544037107001
  0.7511943913730601
 -0.2752062137196889
 -0.23700743414336084

In [54]:
z=myU(U,y)

5-element Array{Float64,1}:
  1.664033297394591
  2.289212897580302
 -0.2195060430755223
 -0.4597332965577868
 -5.369713074556756

In [55]:
x=Pc*z

5-element Array{Float64,1}:
  1.664033297394591
  2.289212897580302
 -5.369713074556756
 -0.4597332965577868
 -0.2195060430755223

In [56]:
# Residual
A*x-b

5-element Array{Float64,1}:
 -6.661338147750939e-16
 -3.885780586188048e-16
 -1.1102230246251565e-16
 -3.3306690738754696e-16
 -3.885780586188048e-16

## Accuracy

Consider the system $Ax=b$, where $A$ is nonsingular.

In order to apply concepts from the notebook 
[L04 Backward Error and Stable Algorithms](L04%20Backward%20Error%20and%20Stable%20Algorithms.ipynb), we need to:

1. develop perturbation theory for the given problem, and
2. analyse errors in the algorithm (Gaussian elimination)

### Perturbation theory

Let

$$
(A+\delta A)\hat x=(b+\delta b)
$$

for some $\hat x=x+\delta x$.

We want to estimate

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

We introduce some notation (see [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \quad \hat x=x(\varepsilon), \quad
\varepsilon\in\mathbb{R},
$$

which yields one-dimensional problem 

$$
(A+\varepsilon F)\,x(\varepsilon)=b+\varepsilon f, 
$$

for some (unknown) matrix $F$ and vector $f$. 

Differentiating with respect to $\varepsilon$ gives

$$
Fx(\varepsilon)+(A+\varepsilon F)\, x(\varepsilon)=f.
$$

Setting $\varepsilon=0$ gives

$$
F x+A\dot x(0)=f,
$$

or

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylor formula arround $\varepsilon=0$ gives

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

that is, by neglecting $O(\varepsilon^2)$ term,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Properties of norm imply

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Finally, since $\| b\| \leq \| A\| \| x\|$, i

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

The number
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

is __condition number__ (or __condition__) the matrix $A$, and it tells us
how much are relative changes in the input data (matrix $A$ and vector $b$) 
relatively amplified in the solution.

Consider the following example (see [Numerička matematika, p. 42][RS04]):


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [57]:
A= [0.234 0.458; 0.383 0.750]

2×2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75

In [58]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [59]:
x=A\b

2-element Array{Float64,1}:
 -1.0000000000002423
  1.0000000000001237

In [60]:
δb=[0.00009; 0.000005]
x₁=A\(b+δb)

2-element Array{Float64,1}:
 -0.24174418604640305
  0.6127906976743631

In [61]:
cond(A), norm(δb)/norm(b), norm(x₁-x)/norm(x)

(11322.197586092605, 0.0002096449170953002, 0.6020311134825742)

In [62]:
δA=[-0.001 0;0 0]
x₂=(A+δA)\b

2-element Array{Float64,1}:
 0.12951807228916615
 0.42319277108433245

In [63]:
cond(A), norm(δA)/norm(A), norm(x₂-x)/norm(x)

(11322.197586092605, 0.0010134105190591591, 0.896804787832142)

### Errors in Gaussian elimination

According to [Matrix Computations, section 3.3][GVL13], the COMPUTED factors
$\hat L$ and $\hat U$ satisfy

$$
\hat L\cdot \hat U = A+\delta A,
$$

where (the inequality is interpreted elementwise, $\varepsilon$ is the machine precision)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Neglecting the $O(\varepsilon^2)$ term and taking norms yields

$$
\|\delta A \| \lesssim O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

so

$$
 \frac{\|\delta A \|}{\|A\|} \lesssim O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

If Gaussian eleimination is computed using row pivoting, then, most probably, the last quotient will be small ($\approx 1$). Further, the error in solving tirangular systems is not larger than this one, so inserting it into (1) it follows that the error in the computed solution satisfies 

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

To conclude:

> _If the condition number is large, the solution may be inaccurate._

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [64]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.6614350593639147
 0.48650890125687063
 0.41728889115919254
 0.38869327482214744
 0.596574907748231
 0.07592661490035035
 0.4860952120877726
 0.46900411658934393
 0.528264975117307
 0.7552505181107274

In [65]:
# Vandermonde matrices are notoriously ill-conditioned.
V=Array{Float64}(undef,n,n)
for i=1:n
    V[:,i]=v.^(i-1)
end
V=V'

10×10 Adjoint{Float64,Array{Float64,2}}:
 1.0        1.0         1.0          …  1.0         1.0         1.0
 0.661435   0.486509    0.417289        0.469004    0.528265    0.755251
 0.437496   0.236691    0.17413         0.219965    0.279064    0.570403
 0.289375   0.115152    0.0726625       0.103164    0.14742     0.430797
 0.191403   0.0560226   0.0303213       0.0483845   0.0778767   0.32536
 0.126601   0.0272555   0.0126527    …  0.0226925   0.0411395   0.245728
 0.0837381  0.01326     0.00527984      0.0106429   0.0217326   0.185586
 0.0553873  0.00645113  0.00220322      0.00499156  0.0114806   0.140164
 0.0366351  0.00313853  0.000919379     0.00234106  0.00606477  0.105859
 0.0242318  0.00152692  0.000383647     0.00109797  0.00320381  0.0799502

In [66]:
bᵥ=rand(n)

10-element Array{Float64,1}:
 0.33104889180804165
 0.8560704147914053
 0.7138667824894915
 0.39559547158529984
 0.2037555212946418
 0.6627753767486095
 0.45341074074282517
 0.23021537565747785
 0.311920548800684
 0.31094083311233645

In [67]:
xᵥ=V\bᵥ

10-element Array{Float64,1}:
    -1.9505175985107422e6
     1.2442697758199965e11
    -1.6094317687911946e8
     3.420574394963029e7
     1.988657200013696e7
 -2127.8676386500338
    -1.2676305033862157e11
     2.7705212826177187e9
    -3.256952028159136e8
 50183.54666973968

In [68]:
cond(V)

1.804967496870472e13

In [69]:
Vbig=map(BigFloat,V)
bbig=map(BigFloat,bᵥ)
xbig=Vbig\bbig;

In [70]:
map(Float64,norm(xbig-xᵥ)/norm(xbig))

5.8453671489411944e-5

### Artificial ill-conditioning

In [71]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A₁=[1e-4 1e-4;1 2]
b₁=[1e-4;3]
x₁=A₁\b₁
x,cond(A₁),x-x₁

(x, cond(A)) = ([-1.0, 2.0], 6.854101966249685)


([-1.0, 2.0], 50000.00017991671, [8.881784197001252e-16, -4.440892098500626e-16])

### Condition estimation

Computing the condition number according to the definition $\kappa(A)=\|A\| \cdot \|A^{-1}\|$ requires the inverse matrix, which requires $O(n^3)$ operacija. That is the same order of magnitude of operations needed to solve the entire system. However, when solving the system, the triangular factors $L$ and $U$ are already at our disposal, which can be used to approximate condition number using just $O(n^2)$ operation. 
Details of this approach can be found in [Matrix Computations, section 3.5.4][GVL13]. 
LAPACK routine 
[dtrcon.f](http://www.netlib.org/lapack/explore-html/d9/d84/dtrcon_8f_source.html) computes approximate reciprocal of the condition number of a triangular matrix.

Let us estimate the condition number of the Vandermonde matrix from the previous example.


[GVL13]: #1 "G. Golub, C. Van Loan,'Matrix Computations', 4th Edition, John Hopkins, baltimore, 2013"  

In [72]:
?LAPACK.trcon!

```
trcon!(norm, uplo, diag, A)
```

Finds the reciprocal condition number of (upper if `uplo = U`, lower if `uplo = L`) triangular matrix `A`. If `diag = N`, `A` has non-unit diagonal elements. If `diag = U`, all diagonal elements of `A` are one. If `norm = I`, the condition number is found in the infinity norm. If `norm = O` or `1`, the condition number is found in the one norm.


In [73]:
L,U=lu(V);

In [74]:
cond(V,1),cond(L,1),cond(U,1)

(1.6503694453980852e13, 37.48682914049708, 6.680315740239664e12)

In [75]:
1 ./LAPACK.trcon!('O','L','U',L),1 ./LAPACK.trcon!('O','U','N',U)

(37.48682914049708, 6.680315740239664e12)

## Residual


The computed solution $\hat x$ of the system $Ax=b$ is the exact solution of a nearby system (see [Afternotes on Numerical Analysis, p. 128][Ste96]):


$$ 
(A+\delta A)\,\hat x=b. \tag{1}
$$

__Residual__ is defined as 

$$
r=b-A\hat x.
$$

Then, 

$$
0=b-(A+\delta A)\,\hat x=r- \delta A\,\hat x. 
$$

Therefore, 

$$ 
\| r\| = \| \delta A\,\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

or

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Thus, if the   __relative rezidual__ 

$$ \frac{r}{\| A\| \cdot \|\hat x \|}$$

has large norm, then  _the solution is not computed stably._

On the other side, if the relative residual is small in norm, then _the solution is computed stably_. Indeed, for

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

(1) holds:

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Also, 

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Let us compute residuals for the previous exmple of dimension $2$:

[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [76]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [77]:
norm(r)/(norm(A)*norm(x))

0.0

In [78]:
r₁=b₁-A₁*x₁

2-element Array{Float64,1}:
 4.0657581468206416e-20
 0.0

In [79]:
norm(r₁)/(norm(A₁)*norm(x₁))

8.131516277378246e-21

Residual for the Vandermonde system:

In [80]:
rᵥ=bᵥ-V*xᵥ

10-element Array{Float64,1}:
 -3.2326922330128127e-6
  6.154785454404177e-6
 -1.954731272624244e-6
  4.324341323247438e-6
 -1.7407817809456105e-7
  2.618816065780294e-6
  8.780382501072381e-7
  1.6795204071939906e-6
 -5.394571205297183e-7
 -5.536515290671673e-8

In [81]:
norm(rᵥ)/(norm(V)*norm(xᵥ))

1.343381577803109e-17

We conclude that the solution $x_v$ is computed stably, that is, with small backward error in the input data. This still does not mean that the relative error in the computed solution is small.